# Calculo de LTV y Cobertura Mensual 

![Guia de Valores Fasecolda](https://autosboutique.com/wp-content/uploads/2022/01/Fasecolda400-300x300.jpg)


*"Los valores de los vehículos de la Guía de valores de Fasecolda, son el resultado de un promedio de oferta de vehículo usado en el país según la oferta disponible en las diferentes fuentes del mercado clasificados, concesionarios e Internet."*


In [82]:
##CARGUE DE LIBRERIAS UTILES Y NECESARIAS
import pyodbc # Importando la librería pyodbc para conectarse a bases de datos ODBC
import pandas as pd # Importando la librería Pandas para trabajar con DataFrames
import numpy as np   # Importando la librería Numpy para operaciones matemáticas y estadísticas
import glob # Importando la librería Glob para obtener listas de archivos basadas en patrones
import re # Importando la librería re para trabajar con expresiones regulares
from alive_progress import alive_bar # Importando la barra de progreso alive_progress para mostrar  barras de progreso animadas en la consola
import datetime # Importando la librería datetime para trabajar con fechas y horas
import os # Importando la librería os para interactuar con el sistema operativo (paths, environment variables, etc.)
import timeit
import time
from datetime import datetime, timedelta, date
import calendar
pd.set_option('display.max_columns', None)

In [83]:
inicio = time.time()
time.sleep(1)

In [84]:
# Funciones

def first_day(d):
    if d.month == 1:
        return d.replace(month =12, year = d.year -1, day= 1)
    else:
        return d.replace(month= d.month -1, year= d.year, day = 1)
    
def last_day(d):
    if d.month == 1:
        return d.replace(month =12, year = d.year -1, day= 31)
    elif d.month == 3:
        if calendar.isleap(d.year) == True:
            return d.replace(month =d.month -1, year = d.year, day= 29)
        else:
            return d.replace(month =d.month -1, year = d.year, day= 28)
    elif (d.month == 5) | (d.month == 7) |(d.month == 10) | (d.month == 12):
        return d.replace(month =d.month -1, year = d.year, day= 30)
    else:
        return d.replace(month= d.month -1, year= d.year, day = 31)


**Se declaran la fecha correpondiente a los dias primero y ultimo del mes inmediatamente anterior**

In [85]:
# hoy = date.today()
# primero = first_day(hoy).strftime('%Y%m%d')
# ultimo = last_day(hoy).strftime('%Y%m%d')
# ayer =datetime.today()+timedelta(days=-1)
primero = 20250101
ultimo = 20250131
ayer = '2025-05-02 14:42:16.563049'
ayer = datetime.strptime(ayer, '%Y-%m-%d %H:%M:%S.%f')
#año
a1 = 2025
print(primero, ultimo, ayer)

20250101 20250131 2025-05-02 14:42:16.563049


**Se importa la base de datos de Fact_Garantias del periodo deseado (recordar que es mes vencido)**

In [86]:
cnxn = pyodbc.connect('DRIVER=SQL Server;Server=192.168.50.38\\DW_FZ;Database=DW_FZ;Port=1433;Trusted_Connection=yes')

Base de garantias 

In [87]:
query = f"""select * 
from (SELECT *
	  ,rank() over (partition by [vehiculo_marca],credito order by id_fecha desc) rank_id
     FROM [DW_FZ].[dbo].Fact_Garantias where id_fecha BETWEEN  {primero} AND {ultimo}) a
  where rank_id=1;
"""

garantias_total = pd.read_sql_query(query, cnxn)
garantias_total.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_4760\1741223245.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  garantias_total = pd.read_sql_query(query, cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46830 entries, 0 to 46829
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id_fecha                   46830 non-null  int64         
 1   credito                    46830 non-null  float64       
 2   codigo_sucursal            46830 non-null  int64         
 3   numero_id_deudor           46830 non-null  float64       
 4   nombre_deudor              46830 non-null  object        
 5   direccion_domicilio        46830 non-null  object        
 6   ciudad                     46830 non-null  object        
 7   telefono_domicilio         46830 non-null  object        
 8   telefono_empresa           46830 non-null  object        
 9   telefono_celular           46830 non-null  object        
 10  valor_desembolso           46830 non-null  float64       
 11  fecha_desembolso           46830 non-null  datetime64[ns]
 12  fech

In [88]:
garantias_total.tail()

,id_fecha,credito,codigo_sucursal,numero_id_deudor,nombre_deudor,direccion_domicilio,ciudad,telefono_domicilio,telefono_empresa,telefono_celular,valor_desembolso,fecha_desembolso,fecha_vencimiento,valor_saldo,dias_mora,valor_garantia,id_vehiculo,vehiculo_placa,vehiculo_marca,vehiculo_descripcion,vehiculo_color,vehiculo_modelo,vehiculo_codigo_fasecolda,vehiculo_cilindraje,vehiculo_numero_motor,vehiculo_numero_chasis,vehiculo_garantia_uso,codigo_garantia,concesionario,tipo_seguro_gps,gps_fecha_vencimiento,vlr_pri_seg_gps,numero_interno_poliza,numero_poliza,tipo_seguro_veh,ind_endo,poliza_fecha_inicio,poliza_fecha_vencimiento,valor_poliza,vlr_pri_seg_veh,memo,fecha_insercion,id_garantia,rank_id
46825,20250131,190549.0,614,900987035.0,ISS SOLUCIONES INTEGRALES SAS,VIA MAMONAL KM 2 calle 6 Ñ56 b -05,BOLIVAR CARTAGE,,127671024,3127671024,266416500.0,2021-04-30,2026-05-05,124507602.0,25,150000000.0,31980,TLL000,VIBROCOMPACTADO Placa TLL000,VIBROCOMPACTADO Marca LIU GONG CLG612H M,AMARILLO,2010,99900209,0,******,******,PAR,1,VEHICOSTA BARRANQUIL,0,1980-01-01,0.0,0.0,,0,0,1980-01-01,1980-01-01,0.0,0.0,,2025-01-31 04:36:39.423,146423,1
46826,20250131,232170.0,414,74795198.0,CHAPARRO FUENTES CARLOS AURELIO,KR 5 4 170 T 2 111,VILLAVICENCIO M,000000098,,3143815408,208769891.0,2024-03-12,2029-03-12,188748074.0,18,608769891.0,1,AVAL,VOLQUETA Placa AVAL,VOLQUETA Marca INTERNATIONAL HV607 SBA H,BLANCO,2024,03626108,0,99186532,3HAEKTAT3RL251729,PUB,1,NAVITRANS,337,2029-03-12,68250.0,654089.0,,91,0,2024-03-12,2029-03-12,608769891.0,0.0,,2025-01-31 04:36:39.423,182045,1
46827,20250131,234193.0,114,13015919.0,PALCHUCAN CHINGAL LUIS HERNANDO,CL 24 C 14 6 CASA 2,IPIALES NARIÑO,136555470,,3136555470,249130000.0,2024-05-08,2029-05-08,249130000.0,22,355900000.0,75779,EQR240,VOLQUETA Placa EQR240,VOLQUETA Marca INTERNATIONAL 7600 WORKST,BLANCO,2019,03626100,0,35351518,3HTWYAHT8KN420979,PUB,1,DIRECTO,337,2029-05-08,68250.0,702278.0,,53,1,2024-08-28,2025-08-28,415900000.0,0.0,,2025-01-31 04:36:39.423,183741,1
46828,20250131,223628.0,314,901362417.0,FERRETERIA Y MATERIALES LM S.A.S.,DG 51 3 OESTE 14,CALI VALLE,,000000000,0000000000,135990000.0,2023-06-26,2029-06-26,113073895.0,0,152000000.0,66761,ETL178,VOLQUETA Placa ETL178,VOLQUETA Marca FOTON AUMARK BJ1065VDJDA-,BLANCO,2023,18926014,0,77122549,LVBDCJD09PN005660,PUB,1,CORBETA S.A.,337,2029-06-26,68250.0,605363.0,,90,0,2023-06-26,2029-06-26,165990000.0,428232.0,,2025-01-31 04:36:39.423,174735,1
46829,20250131,157493.0,114,860055912.0,ECOINSA INGENIERIA S A S,AP CLLE 116 Ñ 9 - 51,BOGOTA D.C.,,102133942,3102133942,210382928.0,2018-11-09,2024-05-09,156807033.0,999,236900000.0,5666,EXZ444,VOLQUETA Placa EXZ444,VOLQUETA Marca HINO FM Modelo 2017 Placa,BLANCO,2017,03726012,0,J08EUB15256,9F3FM8JLTHXX10450,PUB,1,PRACO DIDACOL MORATO,0,1980-01-01,0.0,480733.0,1008388,79,1,2021-02-03,2022-02-03,474300000.0,0.0,,2025-01-31 04:36:39.423,118691,1


In [89]:
garantias_total = garantias_total[["credito", "vehiculo_codigo_fasecolda", "vehiculo_placa", "vehiculo_modelo", "valor_garantia"]]

In [90]:
garantias_total.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46830 entries, 0 to 46829
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   credito                    46830 non-null  float64
 1   vehiculo_codigo_fasecolda  46830 non-null  object 
 2   vehiculo_placa             46830 non-null  object 
 3   vehiculo_modelo            46830 non-null  int64  
 4   valor_garantia             46830 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.8+ MB


Se importan los datos de la basa fasecolda

In [91]:
query = f"""
SELECT [Codigo], [Homologocodigo], [Marca], [Clase], [Referencia1], [Referencia2], [Referencia3], [Um],
[1970], [1971], [1972], [1973], [1974], [1975], [1976], [1977], [1978], [1979], [1980], [1981], [1982], 
[1983], [1984], [1985], [1986], [1987], [1988], [1989], [1990], [1991], [1992], [1993], [1994], [1995], 
[1996], [1997], [1998], [1999], [2000], [2001], [2002], [2003], [2004], [2005], [2006], [2007], [2008], 
[2009], [2010], [2011], [2012], [2013], [2014], [2015], [2016], [2017], [2018], [2019], [2020], [2021], 
[2022], [2023], [2024], [2025], [2026] 
FROM [Analitica].[dbo].[Guia_fasecolda_nuevo_formato];
 ;
""" 
GuiaFasecolda = pd.read_sql_query(query,cnxn)
GuiaFasecolda.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_4760\1636760040.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  GuiaFasecolda = pd.read_sql_query(query,cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17374 entries, 0 to 17373
Data columns (total 65 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Codigo          17374 non-null  object
 1   Homologocodigo  17374 non-null  object
 2   Marca           17374 non-null  object
 3   Clase           17374 non-null  object
 4   Referencia1     17374 non-null  object
 5   Referencia2     17374 non-null  object
 6   Referencia3     17374 non-null  object
 7   Um              17374 non-null  object
 8   1970            17374 non-null  object
 9   1971            17374 non-null  object
 10  1972            17374 non-null  object
 11  1973            17374 non-null  object
 12  1974            17374 non-null  object
 13  1975            17374 non-null  object
 14  1976            17374 non-null  object
 15  1977            17374 non-null  object
 16  1978            17374 non-null  object
 17  1979            17374 non-null  object
 18  1980  

merge de guia_fasecolda con garatias 

In [92]:
# Realizamos el merge entre los DataFrames garantias_total y GuiaFasecolda
df_merged = pd.merge(garantias_total, GuiaFasecolda, left_on="vehiculo_codigo_fasecolda", right_on="Codigo", how="left")
df_merged



,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia,Codigo,Homologocodigo,Marca,Clase,Referencia1,Referencia2,Referencia3,Um,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
0,28860.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31250.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,33810.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34491.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34858.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46825,190549.0,99900209,TLL000,2010,150000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46826,232170.0,03626108,AVAL,2024,608769891.0,03626108,03665034,INTERNATIONAL,VOLQUETA,HV607,SBA HD,AT 10800CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,699200,0,0,760000
46827,234193.0,03626100,EQR240,2019,355900000.0,03626100,03665026,INTERNATIONAL,VOLQUETA,7600,WORKSTAR [375HP],MT TD 6X4 [STD] ABS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,415900,437800,460800,500900,567900,0,0,567900
46828,223628.0,18926014,ETL178,2023,152000000.0,18926014,18965014,FOTON,VOLQUETA,AUMARK,BJ1065VDJDA-F1 [110HP],MT 2800CC TD 4X2 [3.5] AA ABS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,114200,126900,152000,0,0,152000


In [93]:
# Crear una lista con las columnas de los años (1970-2026)
columns_years = [str(year) for year in range(1970, 2027)]

# Crear una nueva columna 'valor_por_ano' para almacenar el valor de la columna correspondiente al año
df_merged["valor_por_ano"] = None

# Iterar sobre las filas del DataFrame combinado
for index, row in df_merged.iterrows():
    # Obtener el año de la columna 'vehiculo_modelo'
    year = str(row['vehiculo_modelo'])  # Convertir el año a string para la comparación
    
    # Verificar si el año está en las columnas de los años
    if year in columns_years:
        # Asignar el valor de la columna correspondiente al año
        df_merged.at[index, 'valor_por_ano'] = row[year]




In [94]:
print(df_merged[["credito","vehiculo_placa", "vehiculo_modelo", "valor_garantia" ,"vehiculo_codigo_fasecolda", "valor_por_ano", ]].head())


   credito vehiculo_placa  vehiculo_modelo  valor_garantia  \
0  28860.0                               0             0.0   
1  31250.0                               0             0.0   
2  33810.0                               0             0.0   
3  34491.0                               0             0.0   
4  34858.0                               0             0.0   

  vehiculo_codigo_fasecolda valor_por_ano  
0                                    None  
1                                    None  
2                                    None  
3                                    None  
4                                    None  


In [ ]:
# Aplicar una depreciación del 25% al valor en la columna 'valor_garantia'
df_merged["valor_garantia_depreciado"] = df_merged["valor_garantia"] * 0.75

# Asegurarnos de que los valores no sean negativos
df_merged["valor_garantia_depreciado"] = df_merged["valor_garantia_depreciado"].apply(lambda x: max(x, 0))

# Imprimir el DataFrame con la nueva columna 'valor_garantia_depreciado'
print(df_merged[["vehiculo_codigo_fasecolda", "vehiculo_modelo", "valor_garantia", "valor_garantia_depreciado"]].head())
# solo quieor que se lo aplique a los que no cruzaron con fasecolda 


  vehiculo_codigo_fasecolda  vehiculo_modelo  valor_garantia  \
0                                          0             0.0   
1                                          0             0.0   
2                                          0             0.0   
3                                          0             0.0   
4                                          0             0.0   

   valor_garantia_depreciado  
0                        0.0  
1                        0.0  
2                        0.0  
3                        0.0  
4                        0.0  


Depresiación

In [102]:
# Crear la nueva columna 'valor_calculado' con la fórmula
df_merged["valor_calculado"] = df_merged.apply(
    lambda row: 0.05 * (a1 - row['vehiculo_modelo']) if a1 > row['vehiculo_modelo'] else 0,
    axis=1
)
# Imprimir el DataFrame con la nueva columna 'valor_calculado'
print(df_merged[["vehiculo_codigo_fasecolda", "vehiculo_modelo", "valor_calculado"]].head())


  vehiculo_codigo_fasecolda  vehiculo_modelo  valor_calculado
0                                          0           101.25
1                                          0           101.25
2                                          0           101.25
3                                          0           101.25
4                                          0           101.25


**Ahora se importan los datos de Cartera Mes**

In [ ]:
df_merged

In [97]:
query = f"""
SELECT id_fecha, credito, id_fecha_desembolso, tipo_id_deudor, numero_id_deudor, id_vehiculo, placa_vehiculo, 
    marca_vehiculo, linea_vehiculo, modelo_vehiculo, plazo_total, valor_desembolso, numero_dias_mora, codigo_oficial, 
    estado_cartera, valor_garantia, saldo_capital, total_deuda, tipo_producto 
FROM DW_FZ.dbo.Fact_Cartera_Mes
WHERE (id_fecha = N'{ultimo}') ;
""" 
cartera_mes = pd.read_sql_query(query,cnxn)
cartera_mes.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_4760\670014921.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cartera_mes = pd.read_sql_query(query,cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42203 entries, 0 to 42202
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_fecha             42203 non-null  object 
 1   credito              42203 non-null  float64
 2   id_fecha_desembolso  42203 non-null  object 
 3   tipo_id_deudor       42203 non-null  int64  
 4   numero_id_deudor     42203 non-null  float64
 5   id_vehiculo          42203 non-null  int64  
 6   placa_vehiculo       42203 non-null  object 
 7   marca_vehiculo       42203 non-null  object 
 8   linea_vehiculo       42203 non-null  object 
 9   modelo_vehiculo      42203 non-null  int64  
 10  plazo_total          42203 non-null  int64  
 11  valor_desembolso     42203 non-null  float64
 12  numero_dias_mora     42203 non-null  int64  
 13  codigo_oficial       42203 non-null  object 
 14  estado_cartera       42203 non-null  object 
 15  valor_garantia       42203 non-null 

**Ahora se importan los datos de Cartera Mes**

Identificacon tipo de credito y garantia 

In [98]:
# Creacion de base con los creditos que tienen garantia hipotecaria

#Lectura de la base que contiene los creditos con garantia hipotecaria
hipotecario = pd.read_excel(r'\\192.168.40.9\garaje$\BackUps_Analitica\BACKUP IVAN\ORGANIGRAMA\LTV\LTV MENSUAL\Hipotecas\HIPOTECAS.xlsx',skiprows=1)
hipotecario = hipotecario[['CREDITO','VALOR AVALÚO','FECHA AVALÚO']] # se toman solo las columnas que resultan utiles para el proceso
hipotecario.info()

hipotecario[['CREDITO','VALOR AVALÚO']].value_counts() #Se verifica los valores que contiene respecto a las varianñes credito y el valor del avaluo

hipotecario['año_avaluo'] = pd.to_datetime(hipotecario['FECHA AVALÚO'],format='mixed')
hipotecario['año_avaluo'] = [hipotecario['año_avaluo'][i].year for i in hipotecario.index]
hipotecario['año_avaluo'] = hipotecario['año_avaluo'].replace(1970,0)
print(hipotecario.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CREDITO       18 non-null     int64         
 1   VALOR AVALÚO  18 non-null     int64         
 2   FECHA AVALÚO  18 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(2)
memory usage: 564.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CREDITO       18 non-null     int64         
 1   VALOR AVALÚO  18 non-null     int64         
 2   FECHA AVALÚO  18 non-null     datetime64[ns]
 3   año_avaluo    18 non-null     int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 708.0 bytes
None
